In [4]:
import pandas as pd
import json
from dotenv import load_dotenv

load_dotenv()
from openai import OpenAI
client=OpenAI()

## Ingestion

In [5]:
df=pd.read_csv('Dog-Cat-QA.csv')
df.head()

,id,Question,Answer
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...


In [6]:
documents = df.to_dict(orient='records')

# data = df.to_dict(orient='records')


In [7]:
documents[0]

{'id': 0,
 'Question': 'What are some examples of breed-typical personalities in dogs?',
 'Answer': 'Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.'}

In [8]:
prompt_template = """
You are simulating a pet owner who's seeking advice about their dog or cat.
Formulate 3 questions this pet owner might ask based on a FAQ record.
The record should contain the answer to the questions, and the questions should be complete 
and not too short. If possible, use as few words as possible from the record.

The record:

question: {Question}
answer: {Answer}

Provide the output in parsable JSON:
{{"questions": ["question1", "question2", "question3"]}}
""".strip()


In [9]:

prompt = prompt_template.format(**documents[0])
prompt

'You are simulating a pet owner who\'s seeking advice about their dog or cat.\nFormulate 3 questions this pet owner might ask based on a FAQ record.\nThe record should contain the answer to the questions, and the questions should be complete \nand not too short. If possible, use as few words as possible from the record.\n\nThe record:\n\nquestion: What are some examples of breed-typical personalities in dogs?\nanswer: Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.\n\nProvide the output in parsable JSON:\n{"questions": ["question1", "question2", "question3"]}'

In [17]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [18]:
json_response=generate_questions(documents[0])

In [10]:
json_response

'{"questions": ["What types of personality traits are commonly found in sporting dog breeds?", "How does breed selection affect a dog\'s personality?", "Can you explain the general demeanor of hound breeds?" ]}'

In [1]:
from tqdm.auto import tqdm


In [15]:
results = {}


In [14]:
len(documents)
# documents[28]

583

In [42]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    try:
        questions=json.loads(questions_raw)    
        results[doc_id] = questions['questions']
    except:
        print(questions_raw)
        print(doc)
        break    

  0%|          | 0/583 [00:00<?, ?it/s]

In [46]:
len(results)

583

In [10]:
import pickle


In [49]:
with open('questions.pkl', 'wb') as f:
    pickle.dump(results, f)

In [12]:
import pickle

# 從檔案讀取 dictionary
with open('questions1.pkl', 'rb') as file:
    loaded_dict = pickle.load(file)

print(len(loaded_dict))
results=loaded_dict

583


In [13]:
final_results=[]
for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))
      

In [14]:
final_results[:10]

[(0,
  'What are the common personality traits associated with sporting dog breeds?'),
 (0, 'Can you explain the typical temperament of hound breeds?'),
 (0,
  "How do breed characteristics influence a dog's behavior and personality?"),
 (1, 'What behavior can I expect from working and herding dog breeds?'),
 (1, 'How do herding dogs usually assess their environment?'),
 (1, 'Can you provide examples of herding behavior in Collie puppies?'),
 (2, 'What dog breeds are most loyal?'),
 (2, 'Which breeds are famous for loyalty?'),
 (2, 'Can you name dogs that exhibit strong loyalty?'),
 (3, 'What protective behaviors do young guarding dogs exhibit?')]

In [15]:
df_result=pd.DataFrame(final_results,columns=['id','question'])

In [16]:
df_result.to_csv('../data/ground-truth-retrieval.csv',index=False)

In [24]:
import minsearch
index=minsearch.Index(
    text_fields=['question'],
    keyword_fields=['id']
)

c:\Users\wufen\.virtualenvs\Pet-Care-RAG-Application-DW-wRTOa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  0%|          | 0/1749 [00:00<?, ?it/s]


NotFittedError: The TF-IDF vectorizer is not fitted